# 常规赛：遥感影像地块分割4月第八名

## 赛题简介

赛题任务
本赛题旨在对遥感影像进行像素级内容解析，并对遥感影像中感兴趣的类别进行提取和分类，以衡量遥感影像地块分割模型在多个类别（如建筑、道路、林地等）上的效果。

数据说明
本赛题提供了多个地区已脱敏的遥感影像数据，各参赛选手可以基于这些数据构建自己的地块分割模型。

训练数据集
样例图片及其标注如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/173cfdc82976423995f4eeea7de014f38c238cd2d5164327860ff80a405a88ea)
![](https://ai-studio-static-online.cdn.bcebos.com/6a64fdeea4544f9f9c99a68dbc3a6fbc5f564c825c5248d18fde668061f7a44f)
![](https://ai-studio-static-online.cdn.bcebos.com/6cedfc4a8aa749668b580546ff6a3ab1edd8dea930464e7e9c9379d574be4d62)
![](https://ai-studio-static-online.cdn.bcebos.com/037ae8d79ae446cda5227a7162c70bacab3da6e958cd4b6ebddc220fd721b4ab)


训练数据集文件名称：train_and_label.zip
包含2个子文件，分别为：训练数据集（原始图片）文件、训练数据集（标注图片）文件，详细介绍如下：

训练数据集（原始图片）文件名称：img_train
包含66,653张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，每张图片的名称形如T000123.jpg。

训练数据集（标注图片）文件名称：lab_train
包含66,653张分辨率为2m/pixel，尺寸为256 * 256的PNG图片，每张图片的名称形如T000123.png。
备注： 全部PNG图片共包括4种分类，像素值分别为0、1、2、3。此外，像素值255为未标注区域，表示对应区域的所属类别并不确定，在评测中也不会考虑这部分区域。

测试数据集
测试数据集文件名称：img_test.zip，详细介绍如下：
包含4,609张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，文件名称形如123.jpg。

## 本项目基于PaddleSeg进行训练与推理

### 首先下载paddleseg，并安装所需的环境

In [ ]:
!git clone https://gitee.com/PaddlePaddle/PaddleSeg
!pip install paddleseg

### 解压数据集并对数据集进行训练集和验证集的划分

In [ ]:
!unzip data/data77571/img_test.zip

### 划分训练集和验证集，由于数据量较多，验证集图片数量划分少一些，按照0.97：0.03划分

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
import shutil
from shutil import copyfile
data_path = "img_train//"
path_origin = "img_train"
label_path = "lab_train"
files = list(filter(lambda x: x.endswith('.jpg'), os.listdir(path_origin)))
random.shuffle(files)
rate = int(len(files) * 0.97)
train_txt = open('train.txt','w')
val_txt = open('val.txt','w')
for i,f in enumerate(files):
    image_path = os.path.join(path_origin, f)
    label_name = f.split('.')[0]+".png"
    target_image = os.path.join(path_origin, f)
    target_label = os.path.join(label_path, label_name)
    if i < rate: 
        train_txt.write(target_image + ' ' + target_label+ '\n')
    else:
        val_txt.write(target_image + ' ' + target_label+ '\n')

In [ ]:
files = list(filter(lambda x: x.endswith('.jpg'), os.listdir("img_testA/")))
print(len(files))

4608


### 模型训练与调优

#### 使用PaddleSeg中ocrnet进行训练，具体训练配置可见configs/ocrnet/remote.yml

In [ ]:
cd PaddleSeg/

/home/aistudio/PaddleSeg


#### 训练

In [ ]:
!python train.py --config configs/ocrnet/remote.yml --do_eval #--resume_model output/best_model

#### 进行测试

#### 由于在paddleseg没有找到保存图片的选项，所以对源代码添加了图片保存，具体如下130-133行，添加文件 PaddleSeg/paddleseg/core/predict.py

![](https://ai-studio-static-online.cdn.bcebos.com/6dff8dc941fc4b0e8342cdc3e6bf43b813c93a915795493fb2808211c16d3658)



In [7]:
!python predict.py --config configs/ocrnet/remote.yml --model_path output/best_model/model.pdparams --image_path ../img_testA/

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-05-09 08:37:17 [INFO]	
---------------Config Information---------------
batch_size: 64
iters: 1600000
learning_rate:
  decay:
    power: 0.9
    type: poly
  value: 0.01
loss:
  coef:
  - 1
  - 0.4
  types:
  - coef:
    - 1
    - 0.01
    losses:
    - type: CrossEntropyLoss
    - type: LovaszHingeLoss
    type: MixedLoss
  - coef:
    - 1
    - 0

#### 压缩结果

In [10]:
!zip result.zip -r output/result/result/